In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

#### **Pesos**

In [7]:
# Define a semente para reprodutibilidade
np.random.seed(42)

# Criando dois cromossomos com 180 pesos
geracao = np.random.rand(2,180)

# Arredondando para duas casas decimais
geracao = np.round(geracao,2)
geracao

array([[0.37, 0.95, 0.73, 0.6 , 0.16, 0.16, 0.06, 0.87, 0.6 , 0.71, 0.02,
        0.97, 0.83, 0.21, 0.18, 0.18, 0.3 , 0.52, 0.43, 0.29, 0.61, 0.14,
        0.29, 0.37, 0.46, 0.79, 0.2 , 0.51, 0.59, 0.05, 0.61, 0.17, 0.07,
        0.95, 0.97, 0.81, 0.3 , 0.1 , 0.68, 0.44, 0.12, 0.5 , 0.03, 0.91,
        0.26, 0.66, 0.31, 0.52, 0.55, 0.18, 0.97, 0.78, 0.94, 0.89, 0.6 ,
        0.92, 0.09, 0.2 , 0.05, 0.33, 0.39, 0.27, 0.83, 0.36, 0.28, 0.54,
        0.14, 0.8 , 0.07, 0.99, 0.77, 0.2 , 0.01, 0.82, 0.71, 0.73, 0.77,
        0.07, 0.36, 0.12, 0.86, 0.62, 0.33, 0.06, 0.31, 0.33, 0.73, 0.64,
        0.89, 0.47, 0.12, 0.71, 0.76, 0.56, 0.77, 0.49, 0.52, 0.43, 0.03,
        0.11, 0.03, 0.64, 0.31, 0.51, 0.91, 0.25, 0.41, 0.76, 0.23, 0.08,
        0.29, 0.16, 0.93, 0.81, 0.63, 0.87, 0.8 , 0.19, 0.89, 0.54, 0.81,
        0.9 , 0.32, 0.11, 0.23, 0.43, 0.82, 0.86, 0.01, 0.51, 0.42, 0.22,
        0.12, 0.34, 0.94, 0.32, 0.52, 0.7 , 0.36, 0.97, 0.96, 0.25, 0.5 ,
        0.3 , 0.28, 0.04, 0.61, 0.5 , 

#### **Clase Neurônio**

In [8]:
class Neuronio:
    def __init__(self,array_entrada = None,array_pesos = None):
        self.entrada = array_entrada
        self.pesos = array_pesos
    
    ## Usando sigmóide na função de ativacao
    def ativacao(self,soma):
        return round(1 /(1 + np.exp(-soma)),2)
    
    def propagation(self):
        # entrada recebendo o valor de entrada para que possa ser alterado
        entrada =  self.entrada.copy()

        # Incluindo um valor 1 no inicío do array da entrada por causa do peso bias
        entrada.insert(0,1)

        soma = 0
        
        # Verifica se os pesos e a entrada tem o mesmo len
        if len(self.pesos) != len(entrada):
            raise ValueError("O número de pesos deve ser igual ao número de entradas, incluindo o bias.")

        # Calculo de soma ponderada a partir do produto escalar entre os vetores de pesos e entrada
        soma = np.dot(entrada, self.pesos)
        
        #     # Obtendo y a partir da função de ativação
        y = self.ativacao(soma)
        return y

    def __str__(self) -> str:
        return "Entrada atual: "+ str(self.entrada) + "\nPesos atuais:" + str(self.pesos)

#### **Classe Rede**

In [9]:
class Rede:    
    def __init__(self,entrada = None,array_pesos= None):
        self.entrada = entrada
        self.oculta = [None]*9
        self.saida = [None]*9
        self.pesos = array_pesos
        
    # Função para fazer o fatiamento dos pesos
    def slice_pesos(self):
        sliced_pesos = []

        # Para cada linha na lista de pesos percorre a linha adicionando a uma nova lista 10 elementos
        for linha in self.pesos:
            for i in range(0,len(linha),10):
                sliced_pesos.append(linha[i: i + 10])

        return sliced_pesos

    ### SETS
    # Função para setar os pesos
    def set_pesos(self,array_pesos):
        self.pesos = array_pesos
        
    def set_entrada(self,entrada):
        self.entrada = entrada

    # Função para setar a camada oculta
    def set_camada_oculta(self):
        lista_neuronios = []
        
        # Obter os pesos fatiados para cada neurônio da camada oculta
        lista_pesos = self.slice_pesos()
        
        # Configurar cada neurônio na camada oculta com uma fatia de pesos
        for i in range(9):  # Para os 9 neurônios
            neuronio = Neuronio(array_entrada=self.entrada, array_pesos=lista_pesos[i])
            self.oculta[i] = neuronio  # Armazena o neurônio na lista oculta
            resultado_propagacao = neuronio.propagation()  # Executa a propagação
            lista_neuronios.append(resultado_propagacao)  # Armazena o resultado
            
        self.oculta = lista_neuronios

    # Função para setar a camada de saida
    def set_camada_saida(self):
        lista_neuronios = []
        
        # Obter os pesos fatiados para cada neurônio da camada oculta
        lista_pesos = self.slice_pesos()
        
        # Configurar cada neurônio na camada oculta com uma fatia de pesos
        for i in range(9):  # Para os 9 neurônios
            neuronio = Neuronio(array_entrada=self.get_oculta(), array_pesos=lista_pesos[i])
            self.saida[i] = neuronio  # Armazena o neurônio na camada de saida
            resultado_propagacao = neuronio.propagation()  # Executa a propagação
            lista_neuronios.append(resultado_propagacao)  # Armazena o resultado
            
        self.saida = lista_neuronios

    def position(self,entrada):
        
        # Set os valores de entrada para a instanciação do neurônio
        self.set_entrada(entrada)
        
        # Setando a camada oculta
        self.set_camada_oculta()
        
        # Setando a camada de saída
        self.set_camada_saida()
        
        # Retornando o index do maior valor do array da camada de saída
        return self.saida.index(max(self.saida))

    ### GETS
    # Função para obter os valores dos pesos
    def get_pesos(self):
        return self.pesos
    
    # Função para obter a camada oculta
    def get_oculta(self):
        return self.oculta
    
    # Função para obter a camada de saida
    def get_saida(self):
        return self.saida
    
    # Print da rede
    def __str__(self) -> str:
        return "Camada oculta: "+ str(self.oculta) + "\nCamada saida:" + str(self.saida) + "\nPesos:" + str(self.pesos)
    

In [10]:
jogada = [1,0,0,0,-1,0,0,0,0]
print(f"Jogada atual: {jogada}")

rede = Rede(array_pesos=geracao)
posicao = rede.position(jogada)
print(f"Melhor posição: {posicao}")

jogada[posicao] = 1
print(f"Jogada após a propagação: {jogada}")

Jogada atual: [1, 0, 0, 0, -1, 0, 0, 0, 0]
Melhor posição: 5
Jogada após a propagação: [1, 0, 0, 0, -1, 1, 0, 0, 0]
